<a href="https://colab.research.google.com/github/mavho/CSE143/blob/master/A2_Part2(check).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Installs the correct packages.

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.

In [0]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    #pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Sentiment Analysis

Loads the data into a train and test set.
X_train is a list of reviews, represented as numpy arrays of integers

In [0]:
(X_train, y_test), (X_valid, y_test) = keras.datasets.imdb.load_data()

row 0, up to 10th column


In [29]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [30]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

1646592/1641221 [==============================] - 0s 0us/step


'<sos> this film was just brilliant casting location scenery story'

Below is an example of how to do custom preprocessing

In [0]:
import tensorflow_datasets as tfds
train_data, dev_data, test_data = tfds.load("imdb_reviews", split=('train[:60%]', 'train[60%:]', 'test'), as_supervised=True)

In [32]:
for X_batch, y_batch in train_data.batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negative



In [0]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [34]:
preprocess(X_batch, y_batch)

(<tf.Tensor: shape=(2, 53), dtype=string, numpy=
 array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie',
         b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
         b'Walken', b'or', b'Michael', b'Ironside', b'Both', b'are',
         b'great', b'actors', b'but', b'this', b'must', b'simply', b'be',
         b'their', b'worst', b'role', b'in', b'history', b'Even',
         b'their', b'great', b'acting', b'could', b'not', b'redeem',
         b'this', b"movie's", b'ridiculous', b'storyline', b'This',
         b'movie', b'is', b'an', b'early', b'nineties', b'US',
         b'propaganda', b'pi', b'<pad>', b'<pad>', b'<pad>'],
        [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
         b'during', b'films', b'but', b'this', b'is', b'usually', b'due',
         b'to', b'a', b'combination', b'of', b'things', b'including',
         b'really', b'tired', b'being', b'warm', b'and', b'comfortable',
         b'on', b'the', b'sette', b'and', b'having', b'j

Construct the vocaulary, counts occurence of each word.

In [0]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in train_data.batch(16).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

Look at only the first 10000 most common words in the vocab

In [0]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

An example of what and how the words are indexed.
This, movie and was was found in the vocab, so the scores are lower, however faaaaantastic wasn't found, so it has a high score?

In [148]:
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b"This movie was faaaaaantastic".split():
    print(word_to_id.get(word) or vocab_size)

22
12
11
10000


replace each word with it's ID

In [0]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [150]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]])>

Creating the final training set.
We batch the reviews, and using the preprocess function, encode them.

In [0]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = train_data.batch(16).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)
dev_set = dev_data.batch(16).map(preprocess)
dev_set = dev_set.map(encode_words).prefetch(1)

What the train set looks like...

In [152]:
for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)
for X_batch, y_batch in dev_set.take(1):
    print(X_batch)
    print(y_batch)

tf.Tensor(
[[   22    11    28   338   287    12   635    25  7697     8    31  1242
   3589    45   490 10966  1806    26    67   156    20     9   196   365
     25    91   145   280     8   447   486    91    67    86    99    24
   6287     9  1609   622   569    22    12     7    28   356  4401  1192
   2053 10325     0     0     0     0     0     0     0     0     0]
 [    6    21    71   523     5   881  1719   370    79    20     9     7
    513   690     5     2  1555     3   228   826    51  1193   127  2054
      4  5007    19     1 10881     4   327    41  6288     2   161   316
     19     9  4402     6  1169  1719    81     1    15    11  1556    14
     85  1170    11  3151  8703     0     0     0     0     0     0]
 [ 3278  6289     1 10808  6290 10367     8     2   790  2101     4  2269
   1095     4  1610  6931   244   750   418    18    52   205   395     5
    102   112   227    19   335     2 10255  1171     1    80     3  5768
   1213 10610     5 10542   882     2

Creates the model and trains it.

In [153]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128,activation ='relu', dropout = .15),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, validation_data=dev_set, epochs=10)

Epoch 1/10
938/938 [==============================] - 94s 100ms/step - loss: 0.5905 - accuracy: 0.6564 - val_loss: 0.5160 - val_accuracy: 0.7578
Epoch 2/10
938/938 [==============================] - 92s 98ms/step - loss: 0.3638 - accuracy: 0.8415 - val_loss: 0.6239 - val_accuracy: 0.7612
Epoch 3/10
938/938 [==============================] - 94s 100ms/step - loss: 0.2339 - accuracy: 0.9098 - val_loss: 0.6773 - val_accuracy: 0.7647
Epoch 4/10
938/938 [==============================] - 98s 105ms/step - loss: 0.1605 - accuracy: 0.9397 - val_loss: 0.7205 - val_accuracy: 0.7516
Epoch 5/10
938/938 [==============================] - 99s 106ms/step - loss: 0.1123 - accuracy: 0.9589 - val_loss: 0.7672 - val_accuracy: 0.7482
Epoch 6/10
938/938 [==============================] - 95s 102ms/step - loss: 0.0747 - accuracy: 0.9733 - val_loss: 1.4886 - val_accuracy: 0.7286
Epoch 7/10
938/938 [==============================] - 94s 100ms/step - loss: 0.0429 - accuracy: 0.9855 - val_loss: 1.6550 - val_acc

In [154]:
history.model.summary()


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, None, 128)         1408000   
_________________________________________________________________
gru_11 (GRU)                 (None, 128)               99072     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 1,507,201
Trainable params: 1,507,201
Non-trainable params: 0
_________________________________________________________________


In [0]:
test_set = test_data.batch(512).map(preprocess)
test_set = test_set.map(encode_words).prefetch(1)



embedds the test data, this is how well it performs?

In [156]:
loss,accuracy = model.evaluate(test_set,steps=10)

10/10 [==============================] - 1s 83ms/step - loss: 2.3921 - accuracy: 0.7258
